In [1]:
# default_exp datasets.genetic_file.Gen

# Genetic file formats

> API details.

In [2]:
#hide
# %load_ext autoreload
# %autoreload 2
from nbdev.showdoc import *
import pandas as pd
from corradin_ovp_utils.catalog import test_data_catalog, conf_test_data_catalog, get_catalog, get_config
from corradin_ovp_utils.datasets.schemas import SingleFilePathSchema
from fastcore.test import ExceptionExpected
import importlib
import corradin_ovp_utils
importlib.reload(corradin_ovp_utils)


INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_input/genetic_file.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_input/sample_file.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_intermediate_output.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_input/genetic_file.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_input/sample_file.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_intermediate_output.yaml


<module 'corradin_ovp_utils' from '/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/corradin_ovp_utils/__init__.py'>

In [258]:
#export
from corradin_ovp_utils.datasets.genetic_file import GeneticFileFormat, triplicate_converter, get_possible_geno_combinations
from typing import Any, Dict, List, Optional, Literal, Union, Protocol
from enum import Enum
import numpy as np
from kedro.io import AbstractVersionedDataSet
from pydantic import BaseModel
from pathlib import Path, PosixPath
from pydantic.dataclasses import dataclass
from dataclasses import InitVar, asdict
from copy import deepcopy
import fsspec
from kedro.io.core import (
    AbstractVersionedDataSet,
    DataSetError,
    Version,
    get_filepath_str,
    get_protocol_and_path,
)
from kedro.extras.datasets.pandas import CSVDataSet
from fastcore.meta import delegates
from fastcore.basics import basic_repr
from functools import partial, wraps, lru_cache
from types import SimpleNamespace
from tqdm.auto import tqdm
import pandas as pd
import itertools
from boltons.strutils import bytes2human
#from dask import delayed
# import dask.dataframe as dd
# import vaex
# import modin.pandas as mpd
#from dask.distributed import Client

In [4]:
# def index_search(list_to_search, query_list):
#     sorter = np.argsort(list_to_search)
#     index = sorter[np.searchsorted(list_to_search, query_list, sorter=sorter)]
#     return index

In [5]:
#hide

def print_with_space(*args, **kwargs):
    print(*args, **kwargs)
    print("------")


In [6]:
test_data_catalog.list()

['genetic_file',
 'genetic_file_common_folder',
 'genetic_file_single',
 'genetic_file_split_by_chrom',
 'genetic_file_bgen',
 'sample_file',
 'geno_each_sample',
 'case_geno_each_sample',
 'control_geno_each_sample']

In [7]:
test_data_catalog.load("genetic_file")._file_path.get_full_file_path()["case"].resolve()

INFO:MainThread:kedro.io.data_catalog:Loading data from `genetic_file` (OVPDataset)...


Path('/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/data/test_data/gen_file/test_CASE_MS_chr22.gen')

In [257]:
#export

class GenFileFormat(GeneticFileFormat):
    prob_n_cols: int
    initial_cols: List[str]
    rsid_col: str
    alleleA_col: str
    alleleB_col: str
    ref_alt_delim: Optional[str]
    pandas_args: Dict[str, Any]
    current_file_path: Optional[str]    
    
    # check that num_cols - initial cols is divisible by prob_n_cols
    # if ref col == alt col then ref_alt_delim needs to be specified, so ref,alt = ref_col.str.split(delim)

    def col_name_generator(self, include_initial_cols=True):
        
        if include_initial_cols:
            for col in self.initial_cols:
                yield (col, "")
        
        if self.sample_ids is None:
            sample_ids_gen = (f"sample{i}" for i in itertools.count(1))
        else:
            sample_ids_gen = (sample_id for sample_id in self.sample_ids)
        
        while True:
            cur_sample_id = next(sample_ids_gen)
            #for allele_prob_col_num in range(1, self.prob_n_cols + 1):
            yield (f"{cur_sample_id}", "AA")
            yield (f"{cur_sample_id}", "AB")
            yield (f"{cur_sample_id}", "BB")
    
    @property
    def gen_info_cols(self):
        return [self.rsid_col, self.ref_col, self.alt_col]
    
    @property
    def column_headers(self):
        #read the first non-header line
        first_line_df = next(self._load_unprocessed(chunksize=1))
        generator = self.col_name_generator()
        col_index = pd.MultiIndex.from_tuples([next(generator) for col in first_line_df.columns], names = ["first", "second"])
        return col_index
    
    @property
    def sample_cols(self):
        cols = [col for col in self.column_headers if col not in set(self.initial_cols)]
        return cols
    
    @property
    def _sample_list(self):
        if self.sample_ids is not None:
            return list(self.sample_ids)
        else:
            return [f"sample{i}" for i in range(1, self.num_samples + 1)] #list(set([col.split("_")[0] for col in self.sample_cols]))
        
    @property
    def num_samples(self):
        return int(len(self.sample_cols)/self.prob_n_cols)
    
    @property
    def load_args(self):
        load_args = deepcopy(self.pandas_args)
        load_args["names"] = self.column_headers
        return load_args
    
    @delegates(pd.read_csv)
    def _load_unprocessed(self, **kwargs):
        return pd.read_csv(filepath_or_buffer = self.current_file_path, **self.pandas_args, **kwargs)
    
    def get_rsid_col(self, chrom =None, **kwargs):
        kwargs_modified = kwargs.copy()
        kwargs_modified["chunksize"] = None
        rsid_col = self._load(chrom = chrom, usecols = [self.rsid_col], **kwargs_modified)
        rsid_col = rsid_col.query("rsid.str.startswith('r')")
        return rsid_col
    
#     def _load(self, chrom = None, **kwargs):
#         # use another attribute so `load` and `_load_unprocessed` can share
#         self.current_file_path = self.get_resolved_file_path(chrom)
#         load_args = self.load_args
#         df = pd.read_csv(self.current_file_path, **load_args, **kwargs)
#         return df

      
    def load_single_chrom_file(self, chrom, **kwargs):
        if chrom == self.file_path.chrom_num:
            return self.load_split_by_chromosome(chrom=chrom, **kwargs)
        else:
            return None
    
    def load_split_by_chromosome(self, chrom, **kwargs):
        resolved_file_path = self.get_resolved_file_path(chrom)
        if not resolved_file_path.is_file():
            print(f"Cannot find file {resolved_file_path.as_posix()}")
            return None
        
        self.current_file_path = self.get_resolved_file_path(chrom)
        sample_cols = self.sample_cols
        sample_list = self._sample_list
        TemplatedGenFileObject = create_file_object_class(self.prob_n_cols, self.rsid_col, alleleA_col= self.alleleA_col, alleleB_col= self.alleleB_col, load_args = self.load_args)
        TemplatedGenFileObject.__repr__ = basic_repr("chrom,file_path")
        #gen_file_object = GenFileObject(df = df, rsid_col = rsid_col, sample_cols = sample_cols, sample_list = sample_list, file_path = self.current_file_path)
        gen_file_object = TemplatedGenFileObject(chrom = chrom,
                                                 sample_cols = sample_cols,
                                                 sample_list = sample_list,
                                                 column_headers= self.column_headers,
                                                 file_path = self.current_file_path)
        return gen_file_object
        
    def load(self, chrom=None, **kwargs):
        if self.is_chrom_template():
            return self.load_split_by_chromosome(chrom = chrom, **kwargs)
        else:
            return self.load_single_chrom_file(chrom = chrom, **kwargs)
    
    def load_all_chrom(self):
        return {chrom: self.load(chrom) for chrom in range(1,23)}
    
    
    def get_geno_each_sample(self, rsid_dict:Dict[int, List[str]], id_col_list: List[str], batch_size=1_000, excluded_sample_ids: List[str] =[]):    
        all_chrom_dict = self.load_all_chrom()
        all_samples_geno_df, all_geno_df = zip(*[all_chrom_dict[chrom].get_geno_each_sample(rsid_list = rsid_list, id_col_list= id_col_list, excluded_sample_ids= excluded_sample_ids) for chrom, rsid_list in rsid_dict.items()])
        return pd.concat(all_samples_geno_df), pd.concat(all_geno_df)
        
            
    
    @delegates(pd.read_csv)
    def get_rsid_df(self,chrom=None, rsid_list=None, **kwargs):
        rsid_df = self.load(chrom = chrom, chunksize=100_000)
        if rsid_list is not None:
            query = f"{self.rsid_col} in {rsid_list}"
            found_rsid_df = rsid_df.query(query)
            
        else:
            found_rsid_df = rsid_df
            
            
#         found_index = found_rsid_df.index
#         found_rsids = set(found_rsid_df[self.rsid_col].unique())
#         found_rsid_df_full = self.load(chrom= chrom, skiprows = lambda x: x not in found_index, **kwargs)
        
        return found_rsid_df
    
    @delegates(load)
    def get_genotypes_df(self, chrom=None, rsid_list: List = None, **kwargs):
        if rsid_list is not None:
            geno_df = self.get_rsid_df(chrom=chrom, rsid_list = rsid_list, usecols = self.gen_info_cols, **kwargs)
        else:
            geno_df = self.load(chrom = chrom, usecols = self.gen_info_cols,**kwargs)
        geno_df["homo_ref"] = geno_df.loc[:,[self.ref_col]] *2
        make_het_geno_func = lambda row: ''.join(sorted([row[self.ref_col],
                                   row[self.alt_col]]))
        geno_df["het"] = geno_df[[self.ref_col, self.alt_col]].apply(make_het_geno_func, axis=1)
        geno_df["homo_alt"] = geno_df.loc[:,self.alt_col] *2
        
        geno_df = geno_df.set_index(self.rsid_col)
        return geno_df
    
    
    @delegates(pd.read_csv)
    def get_rsid_df_big(self, chrom=None, rsid_list= None, **kwargs):
        rsid_col = self.load(chrom=chrom, usecols = [self.rsid_col])
        rsid_col = rsid_list.query("rsid.str.startswith('r')")
        found_rsids = rsid_col.query("rsid in @test_list")
        
        def create_found_rsid_df_generator():
            found_rsid_df_generator = self.load(chrom = chrom, skiprows= lambda x : x not in set(test_result.index), chunksize = 10, index_col = self.rsid_col)
            return found_rsid_df_generator
        
        return create_found_rsid_df_generator()
    
    def sample_columns_iter_list(rsid_df):
        for sample in self._sample_list:
            sample_cols = [f"{sample}_{allele_prob_col_num}" for allele_prob_col_num in range(1, self.prob_n_cols + 1)]
            sample_cols_df = rsid_df[sample_cols]
            yield SimpleNamespace(sample_id = sample,
                                  sample_df = sample_cols_df)
        
        
        
    def get_genotypes_df_big(self, chrom=None, ):
        for rsid_df in found_rsid_df_generator:
            geno_df = get_possible_geno_combinations(rsid_df, big_gen_file.ref_col, big_gen_file.alt_col)
            yield geno_df
    
#     def sample_columns_iter(self, chrom=None, rsid_list=None, **kwargs):
        
#         df = self.get_rsid_df(chrom=chrom, rsid_list = rsid_list, **kwargs)
#         df = df.set_index(self.rsid_col)
        
#         for sample in self._sample_list:
#             sample_cols = [f"{sample}_{allele_prob_col_num}" for allele_prob_col_num in range(1, self.prob_n_cols + 1)]
#             sample_cols_df = df[sample_cols]
#             yield SimpleNamespace(sample_id = sample,
#                                   sample_df = sample_cols_df)
    
#     def sample_columns_iter(
            
    
    #the function has to accept sample_id kwarg
    def apply_func_to_all_samples(self, func, rsid_list=None, chrom=None, **kwargs):
        result_dict = [func(sample_obj.sample_df, sample_id = sample_obj.sample_id, **kwargs) for sample_obj in tqdm(self.sample_columns_iter(chrom= chrom, rsid_list=rsid_list))]#{sample_obj.sample_id: func(sample_obj.sample_df, sample_id = sample_obj.sample_id, **kwargs) for sample_obj in tqdm(self.sample_columns_iter(rsid_list=rsid_list))}
        result_df = pd.concat(result_dict, axis=1)
        #, orient="records")
        return result_df
    
#     def get_geno_each_sample(self, *, chrom=None, rsid_list:List[str]):
#         geno_each_sample_df = self.apply_func_to_all_samples(triplicate_converter, 
#                                         rsid_list = rsid_list,
#                                         genotype_df = self.get_genotypes_df(chrom=chrom, rsid_list = rsid_list),
#                                         chrom=chrom)
#         return geno_each_sample_df.T
    
#     @property
#     def single_line_iter(self, **runtime_kwargs):
#         return pd.read_csv(self.filepath,
#                            **self.load_args,
#                            **runtime_kwargs,
#                            chunksize=1)
    
#     #@delegates(pd.read_csv, but= list(self.load_args.keys()))
#     def load_full(self, **kwargs):
#         return pd.read_csv(self.filepath,
#                            **self.load_args,
#                           **kwargs)
    
#     #if ref_col = alt_col then ref_alt_delim need to be specified

def create_file_object_class(prob_n_cols, rsid_col, alleleA_col, alleleB_col, load_args):
    class GenFileObject(BaseModel):
        chrom: int
        file_path: Path
        #rsid_col: Any
        sample_cols: List
        sample_list: List
        column_headers: pd.MultiIndex
            
        class Config:
            arbitrary_types_allowed = True
        
        @delegates(pd.read_csv, but="names")
        def load_df(self, size_limit = 1_000_000_000, **kwargs): # default limit is 1 GB
            file_size = self.file_path.stat().st_size
            if file_size > size_limit:
                raise MemoryError(f"the file's size ({bytes2human(file_size)}) is too big, input limit is {bytes2human(size_limit)}.\n Please increase the limit or choose a smaller file")
            else:
                df = pd.read_csv(self.file_path, **load_args, **kwargs)
            return df
        
        def sample_columns_iter_list(self, rsid_df: pd.DataFrame, excluded_sample_ids: List[str] =[]):
            excluded_sample_ids_set = set(excluded_sample_ids)
            for sample in self.sample_list:
                if sample in excluded_sample_ids_set:
                    continue
                else:
                    #sample_cols = [f"{sample}_{allele_prob_col_num}" for allele_prob_col_num in range(1, prob_n_cols + 1)]
                    sample_cols_df = rsid_df[sample]
                    yield SimpleNamespace(sample_id = sample,
                                          sample_df = sample_cols_df)
            
        
        def get_geno_each_sample(self, rsid_list: List[int], id_col_list: List[str], batch_size=1_000, excluded_sample_ids: List[str] =[]):
            found_lines = []
            first_level_index = self.column_headers.get_level_values("first")
            id_col_index_list = [first_level_index.get_loc(id_col) for id_col in id_col_list]
            all_samples_df_partials = []
            all_geno_df_partials = []
            query_rsid_set = set(rsid_list)
            
            print(f"reading genetic file and collecting found SNPs for file {self.file_path.as_posix()}")
            
            with open(self.file_path, "r") as file:
                for i,line in tqdm(enumerate(file)):
                    line_list = line.strip().split(" ")
                    for index in id_col_index_list:
                        if line_list[index] in query_rsid_set:
                            line_list.append(line_list[index])
                            found_lines.append(line_list)
                        
                    
                    if len(found_lines) >= batch_size:
                        print("processing batch")
                        all_samples_df_partial, geno_df_partial = self._process_found_lines(found_lines, excluded_sample_ids= excluded_sample_ids)
                        all_samples_df_partials.append(all_samples_df_partial)
                        all_geno_df_partials.append(geno_df_partial)
                        found_lines = []
                        
            print("processing last batch")
            if len(found_lines) > 0:
                all_samples_df_partial, geno_df_partial = self._process_found_lines(found_lines, excluded_sample_ids= excluded_sample_ids)
                all_samples_df_partials.append(all_samples_df_partial)
                all_geno_df_partials.append(geno_df_partial)
                
            all_samples_df = pd.concat(all_samples_df_partials, axis = 1)
            all_geno_df = pd.concat(all_geno_df_partials)
            return all_samples_df, all_geno_df
        
        
        def _process_found_lines(self, found_lines, excluded_sample_ids):
            combined_lines = pd.DataFrame.from_records(found_lines, columns = self.column_headers.append(pd.MultiIndex.from_tuples([("id_col", "")])))
            combined_lines = combined_lines.set_index("id_col")
            geno_df = get_possible_geno_combinations(combined_lines, alleleA_col, alleleB_col)
            all_samples_df = [triplicate_converter(sample.sample_df, genotype_df= geno_df, sample_id = sample.sample_id) for sample in tqdm(self.sample_columns_iter_list(combined_lines, excluded_sample_ids))]
            all_samples_df = pd.concat(all_samples_df, axis=1)
            all_samples_df = all_samples_df.T
            return all_samples_df, geno_df
        
    return GenFileObject


---
### Test functionalities of `GenFileFormat`

In [9]:
conf_test_data_catalog["genetic_file"]["load_args"]

{'prob_n_cols': 3,
 'initial_cols': ['dashes', 'rsid', 'position', 'ref', 'alt'],
 'rsid_col': 'rsid',
 'ref_col': 'ref',
 'alt_col': 'alt',
 'pandas_args': {'sep': ' ', 'header': None}}

In [10]:
conf_test_data_catalog["genetic_file"]["file_path"]["case"]

{'folder': 'data/test_data/gen_file',
 'full_file_name': 'test_CASE_MS_chr22.gen'}

In [11]:
test_genetic_file_single_file_path = Path(conf_test_data_catalog["genetic_file_single"]["file_path"]["folder"])/(conf_test_data_catalog["genetic_file_single"]["file_path"]["full_file_name"])
test_genetic_file_single_file_path

Path('data/test_data/gen_file/test_CASE_MS_chr22.gen')

In [12]:
schema_obj = SingleFilePathSchema(folder=test_genetic_file_single_file_path.parent,
                     full_file_name=test_genetic_file_single_file_path.name)
schema_obj

SingleFilePathSchema(folder=Path('data/test_data/gen_file'), full_file_name='test_CASE_MS_chr22.gen', file_name='test_CASE_MS_chr22', extension='gen', split_by_chromosome=None)

In [13]:
test_genfile_format = GenFileFormat(file_path = schema_obj,
                                    **conf_test_data_catalog["genetic_file"]["load_args"])
test_genfile_format

GenFileFormat(file_path=SingleFilePathSchema(folder=Path('data/test_data/gen_file'), full_file_name='test_CASE_MS_chr22.gen', file_name='test_CASE_MS_chr22', extension='gen', split_by_chromosome=None), prob_n_cols=3, initial_cols=['dashes', 'rsid', 'position', 'ref', 'alt'], rsid_col='rsid', ref_col='ref', alt_col='alt', ref_alt_delim=None, pandas_args={'sep': ' ', 'header': None}, sample_ids=None, current_file_path=None)

In [14]:
test_genfile_format.load() #genotypes()

,dashes,rsid,position,ref,alt,sample1_1,sample1_2,sample1_3,sample2_1,sample2_2,...,sample9769_3,sample9770_1,sample9770_2,sample9770_3,sample9771_1,sample9771_2,sample9771_3,sample9772_1,sample9772_2,sample9772_3
0,---,rs77948203,21249165,G,A,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
1,---,rs1014626,21461017,C,T,0,0.000,1.000,0,0.000,...,1,0,0,1,0,0.000,1.000,0,0,1
2,---,rs9610458,22205353,C,T,0,0.000,1.000,0,0.000,...,0,0,1,0,0,1.000,0.000,0,0,1
3,---,rs5762201,27888455,A,G,0,0.000,1.000,0,0.012,...,1,0,0,1,0,0.000,1.000,0,0,1
4,---,rs1004237,28068501,C,T,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
5,---,rs134490,28730175,C,T,0,0.232,0.768,0,0.014,...,0,0,1,0,0,0.356,0.644,0,0,1
6,---,rs4821519,37102100,G,C,1,0.000,0.000,0,1.000,...,0,1,0,0,1,0.000,0.000,1,0,0
7,---,rs1003500,37262769,C,T,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
8,---,rs5756405,37310954,A,G,0,1.000,0.000,1,0.000,...,1,0,1,0,1,0.000,0.000,0,1,0


In [15]:
test_header = test_genfile_format.column_headers
assert test_header[-1] == 'sample9772_3'

In [16]:
test_genfile_format.get_rsid_df(["rs77948203", "rs9610458"])

In [17]:
next(test_genfile_format.get_rsid_df(["rs77948203", "rs9610458"], chunksize=1))

,dashes,rsid,position,ref,alt,sample1_1,sample1_2,sample1_3,sample2_1,sample2_2,...,sample9769_3,sample9770_1,sample9770_2,sample9770_3,sample9771_1,sample9771_2,sample9771_3,sample9772_1,sample9772_2,sample9772_3
0,---,rs77948203,21249165,G,A,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
1,---,rs1014626,21461017,C,T,0,0.000,1.000,0,0.000,...,1,0,0,1,0,0.000,1.000,0,0,1
2,---,rs9610458,22205353,C,T,0,0.000,1.000,0,0.000,...,0,0,1,0,0,1.000,0.000,0,0,1
3,---,rs5762201,27888455,A,G,0,0.000,1.000,0,0.012,...,1,0,0,1,0,0.000,1.000,0,0,1
4,---,rs1004237,28068501,C,T,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
5,---,rs134490,28730175,C,T,0,0.232,0.768,0,0.014,...,0,0,1,0,0,0.356,0.644,0,0,1
6,---,rs4821519,37102100,G,C,1,0.000,0.000,0,1.000,...,0,1,0,0,1,0.000,0.000,1,0,0
7,---,rs1003500,37262769,C,T,1,0.000,0.000,1,0.000,...,0,1,0,0,1,0.000,0.000,1,0,0
8,---,rs5756405,37310954,A,G,0,1.000,0.000,1,0.000,...,1,0,1,0,1,0.000,0.000,0,1,0


In [18]:
test_genfile_format.get_genotypes_df()

,ref,alt,homo_ref,het,homo_alt
rsid,,,,,
rs77948203,G,A,GG,AG,AA
rs1014626,C,T,CC,CT,TT
rs9610458,C,T,CC,CT,TT
rs5762201,A,G,AA,AG,GG
rs1004237,C,T,CC,CT,TT
rs134490,C,T,CC,CT,TT
rs4821519,G,C,GG,CG,CC
rs1003500,C,T,CC,CT,TT
rs5756405,A,G,AA,AG,GG


In [ ]:
assert test_genfile_format.num_samples == 9772
assert test_genfile_format.gen_info_cols == ['rsid', 'ref', 'alt']

In [ ]:
first_sample = next(test_genfile_format.sample_columns_iter(rsid_list = ["rs9610458", "rs4821519"]))
first_sample.sample_df

In [ ]:
first_sample.sample_id

In [ ]:
triplicate_converter(first_sample.sample_df,
                     sample_id = first_sample.sample_id,
                    genotype_df = test_genfile_format.get_genotypes_df())

In [ ]:
test_genfile_format.get_genotypes_df()

In [ ]:
test_genfile_format.apply_func_to_all_samples(triplicate_converter, 
                                              rsid_list = ["rs9610458", "rs4821519"],
                                              genotype_df = test_genfile_format.get_genotypes_df(["rs9610458", "rs4821519"]))

If you don't input a list of rsids, it will use all the rsids in the dataset, which could take a **LONG TIME**

In [ ]:
test_geno_each_sample = test_genfile_format.apply_func_to_all_samples(triplicate_converter, 
                                              genotype_df = test_genfile_format.get_genotypes_df()).T # transposed

test_geno_each_sample

In [ ]:
test_data_catalog.save("geno_each_sample", test_geno_each_sample)

In [ ]:
test_geno_each_sample[["rs77948203", "rs1014626", "rs1004237"]].reset_index().groupby(["rs77948203", "rs1014626", "rs1004237"])["sample_id"].unique().reset_index() #.melt(value_vars = ["rs77948203", "rs1014626"], ignore_index=False).reset_index().groupby(["rsid", "value"])["index"].unique()

In [ ]:
test_geno_each_sample

In [ ]:
test_genfile = test_genfile_format.load()
test_genfile

---

### Test files split by chromosomes

In [ ]:
test_split_by_chrom = test_data_catalog.load("genetic_file_split_by_chrom")


In [ ]:
assert test_split_by_chrom.files.case.load(chrom=22).shape == (9, 29321)
assert test_split_by_chrom.files.case.get_rsid_df(chrom=22, rsid_list=["rs77948203", "rs1014626"]).shape == (2, 29321)

In [ ]:
with ExceptionExpected(ex=ValueError, regex = "Need to specify `chrom` argument"):
    test_split_by_chrom.files.case.load()
with ExceptionExpected(ex=ValueError, regex = "Need to specify `chrom` argument"):
    test_split_by_chrom.files.case.get_rsid_df(rsid_list=["rs77948203", "rs1014626"])

In [ ]:
test_split_by_chrom.files.case.get_genotypes_df(chrom=22)

In [ ]:
test_split_by_chrom.files.case.get_genotypes_df(chrom=22, rsid_list=["rs77948203", "rs1014626"])

---

## Test big input

In [234]:
big_catalog = get_catalog(env="cluster")
base_catalog = get_catalog(env="base")

INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/cluster/catalog_input/genetic_file.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/cluster/catalog_input/sample_file.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_input/genetic_file.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_input/query_rsids.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_input/sample_file.yaml
INFO:MainThread:anyconfig:Loading: /lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/conf/base/catalog_intermediate_output.yaml


In [235]:
big_genetic_file = big_catalog.load("genetic_file_split_by_chrom")
big_gen_file = GenFileFormat(**vars(big_genetic_file.files.case))
big_gen_file.load(chrom=1,chunksize=1_000, index_col = big_gen_file.rsid_col)

INFO:MainThread:kedro.io.data_catalog:Loading data from `genetic_file_split_by_chrom` (OVPDataset)...


<class '__main__.create_file_object_class.<locals>.GenFileObject'>, chrom = 1, num_samples = 9772, file_path = '/lab/corradin_dbgap/WTCCC2_all_controls_MS_UC/IMPUTE2_ALL/ALL_MS_impute2_chr1.gen'

In [168]:
big_gen_file.num_samples

9772.0

In [184]:
big_gen_file._sample_list[-1]

'sample9772'

In [9]:
big_gen_file.pandas_args

{'sep': ' ', 'header': None}

In [14]:
big_gen_file.file_path.get_full_file_path(22)

Path('/lab/corradin_dbgap/WTCCC2_all_controls_MS_UC/IMPUTE2_ALL/ALL_MS_impute2_chr22.gen')

### Load sample query rsid list

In [27]:
%%time 
rsid_list = big_gen_file.get_rsid_col(chrom = 1)
rsid_list = rsid_list.query("rsid.str.startswith('r')")

CPU times: user 15min 4s, sys: 1min 1s, total: 16min 6s
Wall time: 17min 3s


In [28]:
len(rsid_list)

2149254

In [29]:
test_snps_num = 100_000
test_list = ["fake_rsid"] + list(rsid_list.sample(test_snps_num, random_state =42).values.flatten())
test_result = rsid_list.query("rsid in @test_list")
test_result

,rsid
20,rs145069787
26,rs12723953
33,rs76539137
36,rs72895585
39,rs183954195
...,...
2154981,rs67874053
2154991,rs12728804
2155058,rs145501305
2155062,rs270725


In [213]:
test_result.query("rsid == 'fake_rsid'")

,rsid


In [216]:
base_catalog.save("100k_rsids_only_chrom_1", test_result)
base_catalog.load("100k_rsids_only_chrom_1")

INFO:MainThread:kedro.io.data_catalog:Saving data to `100k_rsids_only_chrom_1` (CSVDataSet)...
INFO:MainThread:kedro.io.data_catalog:Loading data from `100k_rsids_only_chrom_1` (CSVDataSet)...


,rsid
20,rs145069787
26,rs12723953
33,rs76539137
36,rs72895585
39,rs183954195
...,...
2154981,rs67874053
2154991,rs12728804
2155058,rs145501305
2155062,rs270725


In [114]:
test_result.rsid

20         rs145069787
26          rs12723953
33          rs76539137
36          rs72895585
39         rs183954195
              ...     
2154981     rs67874053
2154991     rs12728804
2155058    rs145501305
2155062       rs270725
2155094    rs188420889
Name: rsid, Length: 100000, dtype: object

In [224]:
query_rsid_set = set(test_result.rsid)

---

In [239]:
test_gen_file_chrom_1 = big_gen_file.load(chrom=1,chunksize=1_000, index_col = big_gen_file.rsid_col)
test_gen_file_chrom_1

<class '__main__.create_file_object_class.<locals>.GenFileObject'>, chrom = 1, num_samples = 9772, file_path = '/lab/corradin_dbgap/WTCCC2_all_controls_MS_UC/IMPUTE2_ALL/ALL_MS_impute2_chr1.gen'

In [252]:
%time geno_each_sample_100k_SNPs = test_gen_file_chrom_1.get_rsid_df(query_rsid_set, ["rsid", "position"], batch_size = 100_000)


reading genetic file and collecting found SNPs


0it [00:00, ?it/s]

processing batch


0it [00:00, ?it/s]

processing last batch
CPU times: user 1h 3min 33s, sys: 8min 18s, total: 1h 11min 51s
Wall time: 1h 11min 40s


In [242]:
%time test_gen_file_chrom_1.get_rsid_df(query_rsid_set, ["rsid", "position"], batch_size = 1_000)

reading genetic file and collecting found SNPs


0it [00:00, ?it/s]

processing batch


0it [00:00, ?it/s]

processing batch


...

0it [00:00, ?it/s]

processing batch


0it [00:00, ?it/s]

processing last batch
CPU times: user 2h 17min 41s, sys: 10min 18s, total: 2h 28min
Wall time: 2h 26min 59s


id_col,rs145069787,rs12723953,rs76539137,rs72895585,rs183954195,rs185007189,rs7556636,rs145966388,rs79421110,rs74070515,...,rs1812336,rs12747060,rs115621001,rs72895568,rs2994537,rs67874053,rs12728804,rs145501305,rs270725,rs188420889
sample_id,,,,,,,,,,,,,,,,,,,,,
sample1,GG,TT,AA,CC,TT,AA,GG,AA,GG,CC,...,TT,AA,GG,AA,AA,CC,AA,TT,TT,GG
sample2,GG,CT,AA,CC,TT,AA,GG,AA,GG,CC,...,TT,AG,GG,AA,AA,CC,AG,TT,TT,GG
sample3,GG,NA,AA,CC,TT,AA,GG,AA,NA,CC,...,TT,GG,GG,AA,NA,NA,NA,TT,TT,GG
sample4,GG,NA,AA,CC,TT,AA,GG,AA,GG,CC,...,GT,AG,GG,AA,AA,CC,AG,TT,TT,GG
sample5,GG,CC,AA,CC,TT,AA,GG,AA,GG,CC,...,TT,GG,GG,AA,AA,CC,GG,TT,TT,GG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample9768,GG,CT,AA,CC,TT,AA,GG,AA,AG,CC,...,GT,GG,GG,AA,AA,CC,GG,TT,TT,GG
sample9769,GG,CT,AA,CC,TT,AA,GG,AA,GG,CT,...,TT,GG,GG,AA,AA,CC,GG,TT,TT,GG
sample9770,GG,CC,AA,CC,TT,AA,GG,AA,GG,CC,...,TT,GG,GG,AA,AA,CC,GG,TT,TT,GG


In [195]:
big_gen_file.column_headers.index("rsid")

1

In [144]:
found_line = []
with open(test_generator.file_path, "r") as file:
    for i,line in tqdm(enumerate(file)):
        line_list = line.strip().split(" ")
        if line_list[1] in query_rsid_set:
            found_line.append(line_list)
        
        

0it [00:00, ?it/s]

In [146]:
%time combined_lines = pd.DataFrame.from_records(found_line, columns = big_gen_file.column_headers)
combined_lines = combined_lines.set_index("rsid")

CPU times: user 19min 4s, sys: 59.3 s, total: 20min 3s
Wall time: 20min 3s


In [155]:
%time combined_lines.set_index("rsid")

CPU times: user 47.3 s, sys: 34 s, total: 1min 21s
Wall time: 1min 20s


,dashes,position,ref,alt,sample1_1,sample1_2,sample1_3,sample2_1,sample2_2,sample2_3,...,sample9770_3,sample9771_1,sample9771_2,sample9771_3,sample9772_1,sample9772_2,sample9772_3,homo_ref,het,homo_alt
rsid,,,,,,,,,,,,,,,,,,,,,
rs145069787,---,45001018,G,A,1,0,0,1,0,0,...,0,1,0,0,1,0,0,GG,AG,AA
rs12723953,---,45001453,C,T,0,0,1,0,1,0,...,0,0,1,0,0.052,0.948,0,CC,CT,TT
rs76539137,---,45002271,A,C,1,0,0,1,0,0,...,0,1,0,0,0.002,0.924,0.075,AA,AC,CC
rs72895585,---,45002766,C,T,1,0,0,1,0,0,...,0,1,0,0,1,0,0,CC,CT,TT
rs183954195,---,45003250,T,A,1,0,0,1,0,0,...,0,1,0,0,1,0,0,TT,AT,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs67874053,---,44984011,C,T,1,0,0,1,0,0,...,0,1,0,0,0,1,0,CC,CT,TT
rs12728804,---,44984921,G,A,0,0,1,0,1,0,...,0,1,0,0,1,0,0,GG,AG,AA
rs145501305,---,44991420,T,A,1,0,0,1,0,0,...,0,1,0,0,1,0,0,TT,AT,AA


In [156]:
%time corradin_ovp_utils.datasets.genetic_file.get_possible_geno_combinations(combined_lines.set_index("rsid"), big_gen_file.ref_col, big_gen_file.alt_col)

CPU times: user 1min 9s, sys: 40 s, total: 1min 49s
Wall time: 1min 48s


,homo_ref,het,homo_alt
rsid,,,
rs145069787,GG,AG,AA
rs12723953,CC,CT,TT
rs76539137,AA,AC,CC
rs72895585,CC,CT,TT
rs183954195,TT,AT,AA
...,...,...,...
rs67874053,CC,CT,TT
rs12728804,GG,AG,AA
rs145501305,TT,AT,AA


In [157]:
%time geno_df = corradin_ovp_utils.datasets.genetic_file.get_possible_geno_combinations(combined_lines.set_index("rsid"), big_gen_file.ref_col, big_gen_file.alt_col)
geno_df

CPU times: user 1min 11s, sys: 41.9 s, total: 1min 53s
Wall time: 1min 51s


In [158]:
geno_df

,homo_ref,het,homo_alt
rsid,,,
rs145069787,GG,AG,AA
rs12723953,CC,CT,TT
rs76539137,AA,AC,CC
rs72895585,CC,CT,TT
rs183954195,TT,AT,AA
...,...,...,...
rs67874053,CC,CT,TT
rs12728804,GG,AG,AA
rs145501305,TT,AT,AA


In [ ]:
big_gen_file.sample_cols

In [159]:
def test_func(rsid_df): 
    for sample in big_gen_file._sample_list:
        sample_cols = [f"{sample}_{allele_prob_col_num}" for allele_prob_col_num in range(1, big_gen_file.prob_n_cols + 1)]
        sample_cols_df = rsid_df[sample_cols]
        yield SimpleNamespace(sample_id = sample,
                              sample_df = sample_cols_df)

In [185]:
for sample in tqdm(test_func(combined_lines)):
    print(sample.sample_df)
    print(triplicate_converter(sample.sample_df, genotype_df= geno_df, sample_id = sample.sample_id))
    break

0it [00:00, ?it/s]

            sample1_1 sample1_2 sample1_3
rsid                                     
rs145069787         1         0         0
rs12723953          0         0         1
rs76539137          1         0         0
rs72895585          1         0         0
rs183954195         1         0         0
...               ...       ...       ...
rs67874053          1         0         0
rs12728804          0         0         1
rs145501305         1         0         0
rs270725            0         0         1
rs188420889         1         0         0

[100000 rows x 3 columns]
sample_id   sample1
rsid               
rs145069787      GG
rs12723953       TT
rs76539137       AA
rs72895585       CC
rs183954195      TT
...             ...
rs67874053       CC
rs12728804       AA
rs145501305      TT
rs270725         TT
rs188420889      GG

[100000 rows x 1 columns]


In [187]:
all_samples_df = [triplicate_converter(sample.sample_df, genotype_df= geno_df, sample_id = sample.sample_id) for sample in tqdm(test_func(combined_lines))]

0it [00:00, ?it/s]

In [191]:
%time all_samples_df_final = pd.concat(all_samples_df, axis=1)
all_samples_df_final

CPU times: user 1min 3s, sys: 6.15 s, total: 1min 10s
Wall time: 1min 10s


sample_id,sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample8,sample9,sample10,...,sample9763,sample9764,sample9765,sample9766,sample9767,sample9768,sample9769,sample9770,sample9771,sample9772
rsid,,,,,,,,,,,,,,,,,,,,,
rs145069787,GG,GG,GG,GG,GG,GG,GG,GG,GG,GG,...,GG,GG,GG,GG,GG,GG,GG,GG,GG,GG
rs12723953,TT,CT,NA,NA,CC,CT,NA,TT,NA,CT,...,CT,TT,CT,CC,CT,CT,CT,CC,CT,CT
rs76539137,AA,AA,AA,AA,AA,AA,AA,AA,AA,AA,...,AA,AA,AA,AA,AA,AA,AA,AA,AA,AC
rs72895585,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CC
rs183954195,TT,TT,TT,TT,TT,TT,TT,TT,TT,TT,...,TT,TT,TT,TT,TT,TT,TT,TT,TT,TT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs67874053,CC,CC,NA,CC,CC,CT,CC,CC,CC,CC,...,CC,CC,CC,CC,CC,CC,CC,CC,CC,CT
rs12728804,AA,AG,NA,AG,GG,GG,GG,AG,GG,GG,...,GG,AG,GG,GG,GG,GG,GG,GG,GG,GG
rs145501305,TT,TT,TT,TT,TT,TT,TT,TT,TT,TT,...,TT,TT,TT,TT,TT,TT,TT,TT,TT,TT


In [192]:
%time all_samples_df_final_transposed = all_samples_df_final.T
all_samples_df_final_transposed


CPU times: user 2min 24s, sys: 41.3 s, total: 3min 5s
Wall time: 3min 5s


rsid,rs145069787,rs12723953,rs76539137,rs72895585,rs183954195,rs185007189,rs7556636,rs145966388,rs79421110,rs74070515,...,rs1812336,rs12747060,rs115621001,rs72895568,rs2994537,rs67874053,rs12728804,rs145501305,rs270725,rs188420889
sample_id,,,,,,,,,,,,,,,,,,,,,
sample1,GG,TT,AA,CC,TT,AA,GG,AA,GG,CC,...,TT,AA,GG,AA,AA,CC,AA,TT,TT,GG
sample2,GG,CT,AA,CC,TT,AA,GG,AA,GG,CC,...,TT,AG,GG,AA,AA,CC,AG,TT,TT,GG
sample3,GG,NA,AA,CC,TT,AA,GG,AA,NA,CC,...,TT,GG,GG,AA,NA,NA,NA,TT,TT,GG
sample4,GG,NA,AA,CC,TT,AA,GG,AA,GG,CC,...,GT,AG,GG,AA,AA,CC,AG,TT,TT,GG
sample5,GG,CC,AA,CC,TT,AA,GG,AA,GG,CC,...,TT,GG,GG,AA,AA,CC,GG,TT,TT,GG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample9768,GG,CT,AA,CC,TT,AA,GG,AA,AG,CC,...,GT,GG,GG,AA,AA,CC,GG,TT,TT,GG
sample9769,GG,CT,AA,CC,TT,AA,GG,AA,GG,CT,...,TT,GG,GG,AA,AA,CC,GG,TT,TT,GG
sample9770,GG,CC,AA,CC,TT,AA,GG,AA,GG,CC,...,TT,GG,GG,AA,AA,CC,GG,TT,TT,GG


In [148]:
combined_lines.memory_usage(deep = True).sum()

170551438050

In [111]:
found_dfs = []
query_rsid_set = set(test_result.rsid.tolist())
for df in tqdm(test_generator.df):
    found_rsids = set(df.index.tolist()) & query_rsid_set
    if not found_rsids:
        continue
    else:
        filtered_df = df.loc[found_rsids]
        found_dfs.append(filtered_df)
        print(f"found {filtered_df.shape[0]}")

0it [00:00, ?it/s]

found 54
found 55


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [57]:
big_gen_file._sample_list

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [49]:
def test_func(rsid_df): 
    for sample in big_gen_file._sample_list:
        sample_cols = [f"{sample}_{allele_prob_col_num}" for allele_prob_col_num in range(1, big_gen_file.prob_n_cols + 1)]
        sample_cols_df = rsid_df[sample_cols]
        yield SimpleNamespace(sample_id = sample,
                              sample_df = sample_cols_df)

In [51]:
geno_df = corradin_ovp_utils.datasets.genetic_file.get_possible_geno_combinations(test_double_checked, big_gen_file.ref_col, big_gen_file.alt_col)
geno_df

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,homo_ref,het,homo_alt
rsid,,,
rs145069787,GG,AG,AA
rs12723953,CC,CT,TT
rs76539137,AA,AC,CC
rs72895585,CC,CT,TT
rs183954195,TT,AT,AA
...,...,...,...
rs150084180,TT,AT,AA
rs72671670,TT,CT,CC
rs144804148,TT,CT,CC


In [147]:
sample.sample_df

,sample1_1,sample1_2,sample1_3
rsid,,,
rs182679417,1.000,0.000,0.000
rs141600902,1.000,0.000,0.000
rs200576390,1.000,0.000,0.000
rs145969444,1.000,0.000,0.000
rs184502689,1.000,0.000,0.000
...,...,...,...
rs117795588,1.000,0.000,0.000
rs143237683,1.000,0.000,0.000
rs141475345,0.264,0.736,0.000


In [134]:
test.index.value_counts()

rs150703402    1
rs184623073    1
rs138555012    1
rs139881166    1
rs150639980    1
              ..
rs115567803    1
rs199790570    1
rs184863209    1
rs190308550    1
rs12130529     1
Name: rsid, Length: 9732, dtype: int64

In [143]:
len(set(sample.sample_df.index) - set(geno_df.index))

435

In [142]:
set(sample.sample_df.index) - set(test_result.rsid.unique())

set()

In [140]:
len(set(test_result.rsid.unique()) - set(sample.sample_df.index))

190268

In [35]:
%%time

test_generator = big_gen_file.load(chrom = 1, skiprows= list(set(rsid_list.index) - found_rsid_set), chunksize=10_000, index_col = big_gen_file.rsid_col)
all_SNPs = []
for test in tqdm(test_generator):
    test = test.query("index in @test_result.rsid.unique()")
    geno_df = corradin_ovp_utils.datasets.genetic_file.get_possible_geno_combinations(test, big_gen_file.ref_col, big_gen_file.alt_col)
    all_samples_one_SNP_partition_dfs = [triplicate_converter(sample.sample_df, genotype_df= geno_df, sample_id = sample.sample_id).T for sample in tqdm(test_func(test))]
    all_SNPs.append(pd.concat(all_samples_one_SNP_partition_dfs))
    
    break

0it [00:00, ?it/s]

MemoryError: Unable to allocate 2.18 GiB for an array with shape (29316, 10000) and data type float64

In [158]:
len(all_results)

205212

In [34]:
all_results[10].T

rsid,rs145069787,rs12723953,rs76539137,rs72895585,rs183954195,rs185007189,rs7556636,rs145966388,rs79421110,rs74070515,...,rs77074766,rs78929403,rs3738168,rs3820344,rs185497873,rs150084180,rs72671670,rs144804148,rs78101999,rs79916150
sample_id,,,,,,,,,,,,,,,,,,,,,
sample1007,GG,CC,AA,CC,TT,AA,GG,AA,GG,CC,...,TT,TT,TT,TT,AA,TT,TT,TT,TT,CC


In [25]:
all_results = [result.T for result in tqdm(all_results)]

  0%|          | 0/107492 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
pd.concat(all_results)

MemoryError: Unable to allocate 7.28 GiB for an array with shape (1, 977200000) and data type object

In [127]:
 [triplicate_converter(sample.sample_df, genotype_df= geno_df, sample_id = sample.sample_id) for sample in tqdm(test_func(test)) ]

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
test

In [90]:
test_generator = big_gen_file.load(chrom = 1, skiprows= lambda x : x not in found_rsid_set, chunksize = 10000, index_col = big_gen_file.rsid_col)

In [75]:
test_ddf = dd.read_csv('/lab/corradin_dbgap/WTCCC2_all_controls_MS_UC/IMPUTE2_ALL/ALL_MS_impute2_chr22.gen', **big_gen_file.load_args, assume_missing = True, skiprows= list(set(rsid_list.index) - found_rsid_set), sample=4_000_000)
test_ddf

ValueError: Sample is not large enough to include at least one row of data. Please increase the number of bytes in `sample` in the call to `read_csv`/`read_table`

In [72]:
# all_dfs = []
# for df in tqdm(test_generator):
#     if df.index.item() in found_rsid_set:
#         all_dfs.append(df)

In [91]:
%%time
all_dfs = []
for df in tqdm(test_generator):
    all_dfs.append(df)
    break

0it [00:00, ?it/s]

CPU times: user 4min 49s, sys: 8.56 s, total: 4min 57s
Wall time: 4min 57s


In [86]:
test_df = all_dfs[0]
test_df

,dashes,position,ref,alt,sample1_1,sample1_2,sample1_3,sample2_1,sample2_2,sample2_3,...,sample9769_3,sample9770_1,sample9770_2,sample9770_3,sample9771_1,sample9771_2,sample9771_3,sample9772_1,sample9772_2,sample9772_3
rsid,,,,,,,,,,,,,,,,,,,,,
rs140559483,---,45001033,G,A,1.0,0.0,0.0,1.0,0.0,0,...,0.0,1.000,0.000,0.0,1.0,0.0,0.0,1.0,0.0,0.0
rs111430943,---,45001475,C,T,1.0,0.0,0.0,1.0,0.0,0,...,0.0,1.000,0.000,0.0,1.0,0.0,0.0,1.0,0.0,0.0
rs150368261,---,45002259,C,T,1.0,0.0,0.0,1.0,0.0,0,...,0.0,1.000,0.000,0.0,1.0,0.0,0.0,1.0,0.0,0.0
rs147355609,---,45004037,G,A,1.0,0.0,0.0,1.0,0.0,0,...,0.0,0.513,0.487,0.0,1.0,0.0,0.0,1.0,0.0,0.0
rs75947718,---,45005535,A,G,1.0,0.0,0.0,1.0,0.0,0,...,0.0,1.000,0.000,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs116544829,---,46358224,T,A,1.0,0.0,0.0,1.0,0.0,0,...,0.0,1.000,0.000,0.0,1.0,0.0,0.0,1.0,0.0,0.0
rs116020280,---,46358272,C,T,1.0,0.0,0.0,1.0,0.0,0,...,0.0,1.000,0.000,0.0,1.0,0.0,0.0,1.0,0.0,0.0
rs191993184,---,46358810,G,C,1.0,0.0,0.0,1.0,0.0,0,...,0.0,0.999,0.001,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [45]:
first_sample = next(test_func(test_df))
first_sample

namespace(sample_df=             sample1_1  sample1_2  sample1_3
rsid                                        
rs187800412          1          0          0
rs172018             0          1          0
rs156255             0          0          1
rs78861936           1          0          0
rs112163323          1          0          0
rs117407275          1          0          0
rs139974854          1          0          0
rs192704760          1          0          0
rs270766             0          1          0
rs181242035          1          0          0, sample_id='sample1')

In [89]:
[triplicate_converter(sample.sample_df, genotype_df= geno_df, sample_id = sample.sample_id) for sample in tqdm(test_func(test_df))]

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [163]:
pd.concat(all_dfs)

KeyboardInterrupt: 

In [103]:
index_search(rsid_list.values.flatten(), ["fake_rsid"]  + list(rsid_list.sample(10).values.flatten()))

array([1627547, 1111766,  341437, 1970714, 1105928, 2135379,  240617,
        987551, 1623188,  433210, 1739584])

In [ ]:
rsid_list

In [80]:
big_gen_file_iterator = big_gen_file.load(chrom=22, chunksize=10_000, usecols = [big_gen_file.rsid_col])
big_gen_file_iterator

In [79]:
next(big_gen_file_iterator)#.loc[['rs77948203', 'rs1014626']]

KeyboardInterrupt: 

In [26]:
big_gen_file.load(chrom=22, nrows =10).empty

False

In [33]:
big_gen_file.rsid_col

'rsid'

In [76]:
big_gen_file.load(chrom=22,  skiprows = lambda x: x not in pd.concat(all_filtered).index.values)

,dashes,rsid,position,ref,alt,sample1_1,sample1_2,sample1_3,sample2_1,sample2_2,...,sample9769_3,sample9770_1,sample9770_2,sample9770_3,sample9771_1,sample9771_2,sample9771_3,sample9772_1,sample9772_2,sample9772_3
0,---,rs77948203,21249165,G,A,1,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
1,---,rs1014626,21461017,C,T,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,1


In [71]:
all_filtered = []
for chunk in tqdm(big_gen_file_iterator):
    filtered_df = chunk.query(f"{big_gen_file.rsid_col} in ['rs77948203', 'rs1014626']")
    if filtered_df.empty:
        continue
    else:
        all_filtered.append(filtered_df)

0it [00:00, ?it/s]

In [15]:
%time test = big_gen_file.get_rsid_df(chrom=22, rsid_list=["rs77948203", "rs9610458"])

CPU times: user 9.92 s, sys: 128 ms, total: 10 s
Wall time: 9.97 s


In [ ]:
%time test.compute()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



In [113]:
import dask.dataframe as dd
dd.read_csv(big_gen_file.file_path.get_full_file_path(22), **big_gen_file.load_args, usecols=[big_gen_file.rsid_col])

,rsid
npartitions=377,
,object
,...
...,...
,...
,...


In [8]:
%time 
needed_rsids = test_result.rsid.tolist()
ddf = dd.read_csv(big_gen_file.file_path.get_full_file_path(22), **big_gen_file.load_args, usecols=[big_gen_file.rsid_col]).set_index("rsid")
ddf

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 14.5 µs


NameError: name 'test_result' is not defined

In [124]:
ddf.loc[needed_rsids]

KeyError: 'Cannot index with list against unknown division'

In [ ]:
%time test.compute()


KeyboardInterrupt



In [ ]:
dd.read_csv()

In [38]:
big_genetic_file.files.case.get_rsid_df(chrom=22, rsid_list=["rs77948203", "rs9610458"])

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [25]:
%time big_genetic_file.files.case.load(chrom= 22, usecols =[big_genetic_file.files.case.rsid_col], chunksize=1000)

CPU times: user 2min 7s, sys: 8.66 s, total: 2min 16s
Wall time: 2min 28s


,rsid
0,rs149201999
1,rs146752890
2,rs139377059
3,rs188945759
4,rs6518357
...,...
365639,rs188271599
365640,rs149733995
365641,rs143849140
365642,rs140611932
